# Submission

In [ ]:
import pandas as pd
pd.get_option("display.max_columns")
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor         
from sklearn.metrics import roc_curve

In [ ]:
# To Import Data from google drive (authentication needed)
from google.colab import drive 
drive.mount('/content/gdrive')

Test_Features_Set = pd.read_csv('gdrive/MyDrive/Personal Data Science Projects/Flu Shot Learning/training_set_features.csv')
Test_Data_identifier = Test_Features_Set[['respondent_id']]
df = Test_Features_Set.copy()

In [ ]:
No_of_missing_values_df = df.isnull().sum()

In [ ]:
# List of features having missing values count less than 4000 
# Missing values to be fill with mean or median
list_low_missing = No_of_missing_values_df[ (No_of_missing_values_df.values<4000) & (No_of_missing_values_df.values != 0) ].index.values
list_low_missing

In [ ]:
# List of features having missing values count greater than 4000
# Missing values to be fill with Imputation Methods or will be dropped
list_high_missing = No_of_missing_values_df[ (No_of_missing_values_df.values>4000) & (No_of_missing_values_df.values != 0) ].index.values
list_high_missing

In [ ]:
numerical_features_list = ['household_adults','household_children']

unique_features_list = ['respondent_id']
ordinal_features_list = ['h1n1_concern','h1n1_knowledge','opinion_h1n1_vacc_effective','opinion_h1n1_risk',
                         'opinion_h1n1_sick_from_vacc','opinion_seas_vacc_effective','opinion_seas_risk',
                         'opinion_seas_sick_from_vacc','age_group','education','income_poverty']
categorical_features_list = list( set(df.columns.values)-set(numerical_features_list)-set(unique_features_list)-set(ordinal_features_list) )                        

In [ ]:
list_low_missing = No_of_missing_values_df[ (No_of_missing_values_df.values<4000) & (No_of_missing_values_df.values != 0) ].index.values
list_high_missing = No_of_missing_values_df[ (No_of_missing_values_df.values>4000) & (No_of_missing_values_df.values != 0) ].index.values
numerical_features_list = ['household_adults','household_children']
unique_features_list = ['respondent_id']
ordinal_features_list = ['h1n1_concern','h1n1_knowledge','opinion_h1n1_vacc_effective','opinion_h1n1_risk',
                              'opinion_h1n1_sick_from_vacc','opinion_seas_vacc_effective','opinion_seas_risk',
                              'opinion_seas_sick_from_vacc','age_group','education','income_poverty']
categorical_features_list = list( set(df.columns.values)-set(numerical_features_list)-set(unique_features_list)-set(ordinal_features_list) ) 

In [ ]:
# Ordinal Encoding
feature_name = ordinal_features_list[10]
print(feature_name)
enc1 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan,
                     categories=[[ 'Below Poverty', '<= $75,000, Above Poverty', '> $75,000' ]])
enc1.fit( df[feature_name].values.reshape(-1,1) )
df[feature_name] = enc1.transform( df[feature_name].values.reshape(-1,1)  )

feature_name = ordinal_features_list[9]
print(feature_name)
enc2 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan,
                     categories=[[ '< 12 Years', '12 Years', 'Some College' ,'College Graduate'  ]])
enc2.fit( df[feature_name].values.reshape(-1,1) )
df[feature_name] = enc2.transform( df[feature_name].values.reshape(-1,1)  )

feature_name = ordinal_features_list[8]
print(feature_name)
enc3 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan,
                     categories=[['18 - 34 Years','35 - 44 Years','45 - 54 Years', '55 - 64 Years', '65+ Years']])
enc3.fit( df[feature_name].values.reshape(-1,1) )
df[feature_name] = enc3.transform( df[feature_name].values.reshape(-1,1)  )

In [ ]:
cat_temp = pd.DataFrame()
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
enc.fit( df[categorical_features_list] )
cat_temp[categorical_features_list] = enc.transform( df[categorical_features_list] )

df.drop( columns=categorical_features_list, inplace=True )
df = pd.concat( [df,cat_temp], axis=1 )

In [ ]:
# Impution
features_temp = list( set.intersection( set(categorical_features_list) , set(list_low_missing) ) )
imp_median = SimpleImputer( strategy='median' )
imp_median.fit( df[features_temp] )
df[features_temp] = imp_median.transform( df[features_temp] )

features_temp = list( set.intersection( set(ordinal_features_list) , set(list_low_missing) ) )
imp_median = SimpleImputer( strategy='median' )
imp_median.fit( df[features_temp] )
df[features_temp] = imp_median.transform( df[features_temp] )

features_temp = list( set.intersection( set(numerical_features_list) , set(list_low_missing) ) )
imp_median = SimpleImputer( strategy='median' )
imp_median.fit( df[features_temp] )
df[features_temp] = imp_median.transform( df[features_temp] )

In [ ]:
cat_temp = pd.DataFrame()
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
enc.fit( df[categorical_features_list] )
cat_temp[categorical_features_list] = enc.transform( df[categorical_features_list] )

df.drop( columns=categorical_features_list, inplace=True )
df = pd.concat( [df,cat_temp], axis=1 )

In [ ]:
df.drop(columns='respondent_id', inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
model = XGBClassifier()
imputer_RFR = IterativeImputer( estimator=model, max_iter=3, random_state=0 ) 
imputer_RFR.fit(df)
imputed = imputer_RFR.transform(df)
imputed_df = pd.DataFrame(imputed, columns=df.columns)

df = imputed_df.copy()

In [ ]:
Columns = list( set(categorical_features_list)  ) 
df= pd.get_dummies( df, drop_first=True, dummy_na=False, columns=Columns )

In [ ]:
final_model = Classifier(XGBClassifier())
final_model.fit(df)
final_predictions = final_model.predict_proba(df)

In [ ]:
final_predictions_df = pd.DataFrame()
final_predictions_df[] = final_predictions

In [ ]:
Submission_Data = pd.concat([Test_Data_identifier,final_predictions_df], axis='columns')
Submission_Data

In [ ]:
# from google.colab import files

# Submission_Data.to_csv('BigMartSalesPredictions_Shirsh_Submission_File.csv', encoding = 'utf-8-sig', index=False) 
# files.download('BigMartSalesPredictions_Shirsh_Submission_File.csv')